<center><img src="/files/images/DLI_Header.png" /></center>

# Star Bikes 产品评论分析

在此 notebook 中，您将构建一个 AI 驱动的文档分析器来执行**情感分析**，并为下游任务生成数据。您将学习如何通过为语言模型提供具有指导性的示例来进行少样本学习。

## 学习目标

完成此 notebook 后，您将能够：
* 使用 LLaMA-2 对非结构化文本执行**情感分析**。
* 解释什么是 LLaMA-2 **提示模板**，以及如何在**指令微调**期间使用它。
* 使用**少样本学习**（few-shot learning）指导和提高模型性能。
* 使用 LLaMA-2 生成用于下游任务的 JSON 数据。

## 视频教程

执行以下单元以加载此 notebook 的视频教程。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/03-analyst.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 工作流

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此 notebook 中，我们将使用以下功能来支持与 LLM 的交互。现在大概浏览一下就可以，因为之后使用这些功能时，我们会详细介绍。

### 生成模型响应

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 使用示例创建提示词（少样本学习）

In [ ]:
def prompt_with_examples(prompt, examples=[]):
    """
    Constructs a structured prompt string for language models with instructional examples.

    This function takes an initial prompt and a list of example prompt-response pairs, then 
    formats them into a single string enclosed by special start and end tokens used for 
    instructing the model. Each example is included in the final prompt, which could be 
    beneficial for models that take into account the context provided by examples.

    Parameters:
    - prompt (str): The main prompt to be processed by the language model.
    - examples (list of tuples): A list where each tuple contains a pair of strings 
      (example_prompt, example_response). Default is an empty list.

    Returns:
    - str: A string with the structured prompt and examples formatted for a language model.
    
    Example usage:
    ```
    main_prompt = "Translate the following sentence into French:"
    example_pairs = [("Hello, how are you?", "Bonjour, comment ça va?"),
                     ("Thank you very much!", "Merci beaucoup!")]
    formatted_prompt = prompt_with_examples(main_prompt, example_pairs)
    print(formatted_prompt)
    ```
    """
    
    # Start with the initial part of the prompt
    full_prompt = "<s>[INST]\n"

    # Add each example to the prompt
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

## 数据 – Starlight Cruiser 评价

以下是 Starlight Cruiser 自行车（我们虚构的自行车公司 Star Bikes 的一款产品） 的客户评价。我们将向 LLM 提供这些评价，以执行**情感分析**和其它分析任务。

**中性评论**

In [ ]:
review = """
I recently purchased the Starlight Cruiser from Star Bikes, and I've been thoroughly impressed. \
The ride is smooth and it handles urban terrains with ease. \
However, I did find the seat a bit uncomfortable for longer rides. \
Also, the color options could be better. Despite these minor drawbacks, \
the build quality and the performance of the bike are commendable. It's a good value for the money.
"""

**负面评论**

In [ ]:
review_negative = """
Got the Starlight Cruiser last week, and I'm a bit disappointed. \
The brakes are not as responsive as I'd like and the gears often get stuck. \
The design is good but performance-wise, it leaves much to be desired.
"""

**正面评价**

In [ ]:
review_positive = """
I recently purchased the Starlight Cruiser from Star Bikes, and I've been thoroughly impressed. \
The ride is smooth and it handles urban terrains with ease. \
The seat was very comfortable for longer rides and the color options were great. \
The build quality and the performance of the bike are commendable. It's a good value for the money.
"""

## 情感分析

我们先让模型执行**情感分析**，告诉我们其中一个评论的整体情感。最终，我们希望模型只回复我们一个词，`positive`，`negative` 或 `neutral`。

先提醒一下，下面的单元会给出非常奇怪的输出。

In [ ]:
prompt = f"""
What is the overall sentiment of {review}
"""

print(generate(prompt))

---

我们并不完全清楚为什么模型用上面的提示词给出了完全无用的回复，但这样我们至少明白了迭代提示词的重要性，并再次强调了**精确**（precision）的概念。我们先做一个很小的改动，在提示词最后加一个 `?`，这应该能让模型明白我们是在问一个问题，想得到答案。

In [ ]:
prompt = f"""
What is the overall sentiment of {review}?
"""

print(generate(prompt))

---

我们的 `?` 起了巨大的作用！这个例子就提醒了我们，对提示词一个细微的调整，有时都会导致模型响应发生巨大变化。

鉴于我们的目标是从模型中获取单个词的响应，让我们迭代提示词，以便更**具体**（specific）的说明我们想要的响应。

In [ ]:
prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "positive", "negative", or "neutral".
"""

print(generate(prompt))

---

该模型依然比我们期望的更热心，提供了远多于一个词的回复。让我们再次为提示词提供一些**提示**线索来迭代。

In [ ]:
prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "positive", "negative", or "neutral".
Overall Sentiment: 
"""

print(generate(prompt))

---

现在效果好多了。但是，我觉得这条评论应该是 "neutral" 而不是 "positive"。您可以再参考一下评论内容：

In [ ]:
review = """
I recently purchased the Starlight Cruiser from Star Bikes, and I've been thoroughly impressed. \
The ride is smooth and it handles urban terrains with ease. \
However, I did find the seat a bit uncomfortable for longer rides. \
Also, the color options could be better. Despite these minor drawbacks, \
the build quality and the performance of the bike are commendable. It's a good value for the money.
"""

为了再次强调细微的更改会严重影响模型行为，我们稍微修改一下提示词中的选项顺序。

In [ ]:
# NOTE: 'Choose one of "neutral", "negative", or "positive".' is in a different order than the prompt immediately above
prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "neutral", "negative", or "positive".
Overall Sentiment: 
"""

print(generate(prompt))

---

我们最后再换一次顺序看看。

In [ ]:
prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "negative", "neutral", or "positive".
Overall Sentiment: 
"""

print(generate(prompt))

---

现在我们还没有信心用这个提示词得到模型有意义的回复。为了获得更可靠、可信的响应，我们将注意力转向一项重要的技巧，即**少样本学习**，这将允许我们用几个示例指导模型的行为。

## 提供示例：少样本学习

根据所提供示例的数量，以下技巧被称为**单样本学习**、**双样本学习**、**三样本学习**（等等）、**多样本学习**和**一到多样本学习**。在每种情况下，**样本**都是提供给模型的提示/响应示例，来指导其行为。

这些样本通常放在我们真正要问模型的问题之前。根据所使用的模型，有一些特定的格式化样本的方法，来帮助模型理解我们所提供的是提示词/响应示例。

## 指令微调（聊天）模型

如果您看过 HuggingFace 这类模型库，可能会注意到[一些模型后面带有 `-chat` 标志](https://huggingface.co/models?sort=trending&search=meta-llama%2FLlama-2-13b)。这些模型（比如 `Llama-2-13b-chat`）是在基础预训练模型（比如 `Llama-2-13b`）基础上进行额外训练的，通常是为了更好地遵循指令以支持聊天类应用。也就是说，它们已经过了**指令微调**。

虽然预训练 LLM 基础模型通常很擅长根据给定输入下一步的概率来生成文本，但仅仅提供下一个最有可能的文本跟回答问题或遵循指令还是不太一样。

在**指令微调**期间，模型基于用户和模型之间的大量示例交互进行训练，这样模型就能学到如何在聊天对话的上下文中正确遵循指令或做出适当响应。

## LLaMA-2 提示模板

不同指令微调模型（即聊天变体）的示例交互将采用不同的格式。训练过程中使用的模板被称为**提示模板**。通常您可以在给定模型的文档中找到其提示模板。

下面是 LLaMA-2 的提示模板。实际上这是一个稍微简化了的版本，我们稍后会讨论到它缺少了的部分。

```python
<s>[INST] {{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>
```

我们来剖析一下这个**提示模板**。
* 单轮用户/模型交互在 `<s>` 和 `</s>` 标签之间。
* 用户/模型交互的用户部分在 `[INST]` 和 `[/INST]` 标签之间。
* 用户/模型交互的模型部分在 `[/INST]` 标签之后，并以交互结束的标签 `</s>` 结尾。

在**指令微调**期间，模型接受了很多按照**提示模板**格式构造的用户/模型交互示例。考虑到这一点，我们可以利用其训练过程，使用相同的**提示模板**提供我们的指令示例。

## 提供指令示例

下面的 `prompt_with_examples` 函数将帮助我们使用 LLaMA-2 提示模板构建包含指令示例的提示词。

In [ ]:
def prompt_with_examples(prompt, examples=[]):
    """
    Constructs a structured prompt string for language models with instructional examples.

    This function takes an initial prompt and a list of example prompt-response pairs, then 
    formats them into a single string enclosed by special start and end tokens used for 
    instructing the model. Each example is included in the final prompt, which could be 
    beneficial for models that take into account the context provided by examples.

    Parameters:
    - prompt (str): The main prompt to be processed by the language model.
    - examples (list of tuples): A list where each tuple contains a pair of strings 
      (example_prompt, example_response). Default is an empty list.

    Returns:
    - str: A string with the structured prompt and examples formatted for a language model.
    
    Example usage:
    ```
    main_prompt = "Translate the following sentence into French:"
    example_pairs = [("Hello, how are you?", "Bonjour, comment ça va?"),
                     ("Thank you very much!", "Merci beaucoup!")]
    formatted_prompt = prompt_with_examples(main_prompt, example_pairs)
    print(formatted_prompt)
    ```
    """
    
    # Start with the initial part of the prompt
    full_prompt = "<s>[INST]\n"

    # Add each example to the prompt
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

## 单样本学习示例

让我们暂时把情感分析任务放一放，先用简单的文本生成提示词来探索如何用 `prompt_with_examples` 函数来提供一个指令示例，或者换句话说，进行**单样本学习**（one-shot learning）。

In [ ]:
example_prompt = "Give me an all uppercase color that starts with the letter 'O'."
example_response = "ORANGE"

# The function expects prompt/response pairs to be 2-tuples
example_1 = (example_prompt, example_response)

# The function expects all prompt/response 2-tuples to be in a list
examples = [example_1]

# This is the "main" prompt we actually want the model to respond to
prompt = "Give me an all uppercase color that starts with the letter 'P'."

# Use `prompt_with_examples` to create a one-shot learning prompt, with a single example prepended to our main prompt
prompt_with_one_example = prompt_with_examples(prompt, examples)

In [ ]:
print(prompt_with_one_example)

---

`prompt_with_one_example` 包含单轮用户/模型交互（`<s>...</s>`），在主提示词之前使用 LLaMA-2 **提示模板**。请注意，主提示词仅包含交互的用户部分（`[INST]` 和 `[/INST]` 标签之间），剩下的部分留给模型去完成（模型的响应和 `</s>` 标签）。

在使用 `prompt_with_one_example` 之前，我们先来看看仅输入主提示词而不加上指令示例的话模型会怎么响应。

In [ ]:
print(generate(prompt))

---

可以看到，在我们期待的 `PURPLE` 之前还有一些额外的聊天似的响应。

接下来试试 `prompt_with_one_example`，其中包含一个示例，为说明模型应该仅回答我们感兴趣的颜色词汇。

In [ ]:
print(generate(prompt_with_one_example))

## 带示例的情感分析

回到情感分析任务来，之前我们对模型是否会正确标出中性评论缺乏信心，下面让我们应用刚刚学习的**单样本学习**，为模型提供一个人类认为是中性评论的指令示例。

这是一个应该被归为中性评论的示例，其中显然不包含对自行车的正面或负面情感。

In [ ]:
example_neutral_review = """
I've had the chance to put several miles on my new Starlight Cruiser from Star Bikes. 
First off, the bike's design is sleek, and it provides an exceptionally stable ride, 
even when navigating the bustle of city streets. The gear shifting is fluid, 
and the bike feels robust, promising longevity. On the downside, the braking system, 
while reliable, lacks the responsiveness I've experienced with other bikes. 
I also noticed that the handlebar grips can become rather uncomfortable on prolonged journeys. 
Nevertheless, these issues aside, the bike offers impressive performance for its price range, 
making it a solid, middle-of-the-road choice for both commuting and leisure rides.
"""

我们将用这个示例评论来构建可以传给 `prompt_with_examples` 函数的 `examples` 列表。

In [ ]:
example_prompt_neutral = f"""
What is the overall sentiment of this review {example_neutral_review}?

Choose one of "negative", "neutral", or "positive".
Overall Sentiment: 
"""
example_response_neutral = "neutral"

example_neutral = (example_prompt_neutral, example_response_neutral)
examples = [example_neutral]

现在来构建主提示词，再次使用上面的评论，我们希望将其归为中性。

In [ ]:
prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "negative", "neutral", or "positive".
Overall Sentiment: 
"""

prompt_with_one_example = prompt_with_examples(prompt, examples)

In [ ]:
print(generate(prompt_with_one_example))

---

遗憾的是，模型仍将该评价标记为 `"Positive"` 。

## 双样本学习

通常，当**单样本学习**不足以获得我们想要的行为时，可以添加更多示例来进一步约束模型的行为。

在我们的例子中，除了为模型提供中性示例外，再提供一个正面的评论示例，看看它是否能更清楚的理解两者的区别。

In [ ]:
example_review_positive = """
I've been absolutely delighted with my Starlight Cruiser purchase from Star Bikes. 
The bike exudes a charm with its sleek design that turns heads as I glide through city lanes. 
It's not just about looks though; the bike performs wonderfully. The gears shift like a dream, 
making for a ride that's as smooth as silk across various urban terrains. I was initially skeptical 
about the comfort of the seat, but it proved to be pleasantly supportive, even on my longer weekend adventures. 
While the color choices were limited, I found one that suited my style perfectly. 
Any minor imperfections pale in comparison to the bike's overall quality and the sheer joy it brings to my daily commutes. 
For the price, the Starlight Cruiser is an undeniable gem that I would happily recommend.
"""

In [ ]:
example_prompt_positive = f"""
What is the overall sentiment of this review {example_review_positive}?

Choose one of "negative", "neutral", or "positive".
Overall Sentiment: 
"""
example_response_positive = "positive"

In [ ]:
main_prompt = f"""
What is the overall sentiment of this review {review}?

Choose one of "negative", "neutral", or "positive".
Overall Sentiment: 
"""

### 练习：执行双样本学习

执行**双样本学习**，先为模型提供一个中性和一个正面的交互示例，然后再让它响应我们希望分类为 `neutral` 的 `review`。
* 用上面已经定义的 `example_neutral` 作为一个示例。
* 用 `example_review_positive`，`example_prompt_positive` 和 `example_response_positive` 构建一个正面评价的用户/模型交互示例。
* 用这两个示例（中性和正面）和 `main_prompt` 构建一个包含两个示例的提示词（利用 `prompt_with_examples` 函数）。
* 使用您构建的带有两个示例的提示词生成并打印模型响应。

如果遇到问题，可以查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
example_positive = (example_prompt_positive, example_response_positive)
examples = [example_neutral, example_positive]

prompt_with_two_examples = prompt_with_examples(main_prompt, examples)

print(generate(prompt_with_two_examples))

## 生成数据供下游使用

我们的模型现在能有效的进行**情感分析**，我们来扩展其分析功能，让它生成供下游使用的包括正面和负面关键词的 JSON 对象。

我们开始迭代提示词，先要求模型分别列出评论中的正面负面关键词。

In [ ]:
prompt = f"""
From the review, list the positive points and negative points separately: {review}
"""

print(generate(prompt))

---

模型做得很不错。现在我们迭代一下，试试让模型生成 JSON 对象。

In [ ]:
prompt = f"""
From the review, list down the positive points and negative points separately, in JSON: {review}
"""

print(generate(prompt))

---

好像没什么变化。让我们在提示词中更**准确**的说明希望如何格式化数据。（注意：我们必须使用双大括号 `{{`、`}}` 而不是单大括号 `{`、`}`，因为我们使用的是 Python f-string，它会将单个大括号解释为 Python 值的占位符。）

In [ ]:
prompt = f"""
From the review below, list down the positive points and negative points separately, in JSON. Use the following format:

{{"positive": [], "negative": []}}

Review: {review}
"""

print(generate(prompt))

---

这似乎也没有带来多大改变。

## 练习：成功生成 JSON

运用您迄今为止所学到的知识，成功完成这个任务，让模型能输出 JSON。如果您想为模型提供指令示例，我们已在下方为您提供了两个评论示例及其对应的 JSON 输出。

以及下面还有一个接收 LLM 输出的 `pretty_print_json` 辅助函数，如果 LLM 响应是有效的 JSON 格式，该函数将以漂亮的缩进方式打印出该响应。

如果您遇到问题，下面有两种解决方案。它们是几个隐藏单元，您可以单击 `+ N cells hidden` 按钮来展开它们。

In [ ]:
example_reviews = [
"""\
I recently purchased the Starlight Cruiser from Star Bikes, and I've been thoroughly impressed. \
The ride is smooth and it handles urban terrains with ease. \
However, I did find the seat a bit uncomfortable for longer rides. \
Also, the color options could be better. Despite these minor drawbacks, \
the build quality and the performance of the bike are commendable. It's a good value for the money.\
""",
"""\
Got the Starlight Cruiser last week, and I'm a bit disappointed. \
The brakes are not as responsive as I'd like and the gears often get stuck. \
The design is good but performance-wise, it leaves much to be desired.\
"""
]

In [ ]:
example_outputs = [
    {
        "positive": ["smooth ride", "ease of handling urban terrains", "good value for the money"],
        "negative": ["seat uncomfortable for longer rides", "limited color options"]
    },
    {
        "positive": ["good design"],
        "negative": ["brakes not repsonsive", "gears often get stuck", "performance leaves much to be desired"]
    }
]

In [ ]:
def pretty_print_json(json_string):
    print(json.dumps(json.loads(json_string), indent=4))

### 解决方案 1

创建一个提示词/响应的2元组示例列表。请注意，在该实现中，我们只用评论作为提示词，没有给任何其它输入。

In [ ]:
examples = [(example_review, json.dumps(example_output)) for example_review, example_output in zip(example_reviews, example_outputs)]

检查示例的格式。

In [ ]:
examples

---

使用 `review` 作为提示词。

In [ ]:
prompt = review

生成响应。

In [ ]:
json_response = generate(prompt_with_examples(prompt, examples))

In [ ]:
pretty_print_json(json_response)

### 解决方案 2

*解决方案 1* 展示了**双样本学习**的有效性，值得一提的是，我们还可以通过添加一个**提示**线索 `JSON:` 来达到理想的效果。

In [ ]:
prompt = f"""
From the review below, list down the positive points and negative points separately, in JSON. Use the following format:

{{"positive": [], "negative": []}}

Review: {review}
JSON:
"""

pretty_print_json(generate(prompt))

## 关键概念回顾

此 notebook 中介绍了以下关键概念：
* **情感分析**：识别一段文本的情绪或情感。
* **指令微调**：通过基于示例的定制学习提高模型的任务性能。
* **LLaMA-2 提示模板**：预先设计的指导 LLaMA-2 模型响应的格式，用于进行指令微调。
* **少样本学习**：为模型预先提供一或多个指令示例，以改进其响应。

## 可选的进阶练习

如果您想超出本课程的内容进阶一下，可以试试下面的额外开放式练习。

### 多种输出数据格式

我们已经成功生成了 JSON。尝试生成不同形式的数据，比如 HTML 表格。

### 使用 7B 模型

在 notebook 顶部，按照下面的代码重启内核后，取消注释以使用 7B 模型，而不是 13B 模型。试试通过提示工程在使用小（更弱）模型的情况下获得满意的结果。

### 复用模型进行其它分析

LLaMA-2 这类 LLM 的超能力之一是，能够执行过去需要许多模型去处理的多种任务。

试想一下，如果您已经为许多评论收集了“正面”和“负面”关键词，可能会发现许多相似的项目，但模型会记录为不同的字符串。就比如“不错的轮胎”和“很棒的轮胎”。您能否复用模型来创建数据，捕获到被我们识别为同类的项目？

您还有兴趣执行哪些其它的分析？

重启内核
----




为下一个 notebook 释放 GPU 显存，请运行以下单元。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)